# Stock List and Info

## Import

Libraries

In [1]:
import requests
import os
import pandas as pd
import numpy as np
import json

Definitions

In [2]:
# current dir + data
download_dir = os.path.join(os.getcwd(), '../data')

## Get ASX List

In [3]:
# todo: does this work tmr
# The URL you provided
url = "https://asx.api.markitdigital.com/asx-research/1.0/companies/directory/file?access_token=83ff96335c2d45a094gics_df02a206a39ff4"

# Download the file
response = requests.get(url)

# Save the file if the request is successful
if response.status_code == 200:
    with open(f"{download_dir}/asx_companies_directory.csv", "wb") as file:
        file.write(response.content)
    print("File downloaded successfully!")
else:
    print(f"Failed to download file. Status code: {response.status_code}")

File downloaded successfully!


## Get gics sector and industry

In [ ]:
import yfinance as yf

# Create a dataframe to store stock information
stock_data = []

for ticker in ASX_codes:
    ticker += ".AX"
    stock = yf.Ticker(ticker)
    info = stock.info
    stock_data.append({
        'Ticker': ticker,
        'Name': info.get('shortName', 'N/A'),
        'Market Cap': info.get('marketCap', 'N/A'),
        'Sector': info.get('sector', 'N/A'),
        'Industry': info.get('industry', 'N/A'),
    })

# Convert the data into a DataFrame
gics_df = pd.DataFrame(stock_data)

# Display the DataFrame
display(gics_df)

In [ ]:
gics_df.to_csv('../data/asx_gics.csv', index=False)